# Data visualization of stocks

In [6]:
import numpy as np
import glob
import pandas as pd
from bokeh.io import output_notebook
from ipynb.fs.full.library import plot_candlestick

output_notebook()

Loading BokehJS ...

## Find proper file based on symbol parameter

In [3]:
all_files = glob.glob(r"..\data\*.mst")
symbol = "XTB."
found_files = []
for path in all_files:
    if symbol in path:
        found_files.append(path)
found_files

['..\\data\\XTB.mst']

### Bulk plotter

In [2]:
observed = ["CDPROJEKT", "CCC.", "11BIT", "XTB", "MERCATOR", "BOOMBIT", "BLOOBER", "AIRWAY"]
all_files = glob.glob(r"..\data\*.mst")
path_list = []
for item in observed:
    for path in all_files:
        if item in path and "RCF" not in path:
            path_list.append(path)
path_list

['..\\data\\CDPROJEKT.mst',
 '..\\data\\CCC.mst',
 '..\\data\\11BIT.mst',
 '..\\data\\XTB.mst',
 '..\\data\\NEXTBIKE.mst',
 '..\\data\\MERCATOR.mst',
 '..\\data\\BOOMBIT.mst',
 '..\\data\\BLOOBER.mst',
 '..\\data\\AIRWAY.mst']

In [7]:
path_basked = ['..\\data\\CDPROJEKT.mst',
               '..\\data\\11BIT.mst',
               '..\\data\\XTB.mst',
               '..\\data\\MERCATOR.mst',
               '..\\data\\BLOOBER.mst',
               '..\\data\\BOOMBIT.mst',
               '..\\data\\COLUMBUS.mst',
               '..\\data\\AIRWAY.mst']
date_start = "20200301"

for path in path_basked:
    data = pd.read_csv(path, index_col=1)
    data.index = pd.to_datetime(data.index, format="%Y%m%d")
    plot_candlestick(data, date_start, 11, 15, 10)